In [7]:
import pandas as pd
import numpy as np
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Bidirectional, LSTM, Dropout
from keras.optimizers import Adam
from keras_contrib.layers import CRF
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

ImportError: cannot import name 'pad_sequences' from 'keras.preprocessing.sequence' (C:\Users\SlideSciences\AppData\Roaming\Python\Python39\site-packages\keras\preprocessing\sequence.py)

In [ ]:
# Load the data from the Excel sheet
data = pd.read_excel("train_data.xlsx")

# Split the data into features and labels
X = data.iloc[:, 0].values
y = data.iloc[:, 1:].values

# Encode the labels using one-hot encoding
label_encoder = LabelEncoder()
y = [label_encoder.fit_transform(labels) for labels in y]
y = np.array([to_categorical(labels, num_classes=2) for labels in y])

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert the input data to sequences of integers
tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(X_train)
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

# Pad the sequences to ensure that they are all of the same length
max_len = 100
X_train_seq = pad_sequences(X_train_seq, maxlen=max_len)
X_test_seq = pad_sequences(X_test_seq, maxlen=max_len)

# Define the model architecture
model = Sequential()
model.add(tf.keras.layers.Input(shape=(max_len,)))
model.add(tf.keras.layers.Embedding(input_dim=len(tokenizer.word_index)+1, output_dim=128, input_length=max_len))
model.add(Bidirectional(LSTM(128, return_sequences=True)))
model.add(Dropout(0.2))
model.add(Bidirectional(LSTM(64)))
model.add(Dropout(0.2))
model.add(Dense(2, activation="softmax"))

# Compile the model with suitable loss function and optimizer
model.compile(loss="categorical_crossentropy", optimizer=Adam(lr=0.001), metrics=["accuracy"])

# Train the model on the data
model.fit(X_train_seq, y_train, batch_size=32, epochs=10, validation_split=0.2)


In [1]:
import tensorflow as tf